# 深圳各区二手房指导价明细与汇总

In [ ]:
import pandas as pd

In [6]:
import os

In [7]:
import re

In [3]:
print(os.path.exists('./txt/szhouse.txt'))

True


In [20]:
data_list = []

In [13]:
with open('./txt/szhouse.txt', mode='r', encoding='utf-8') as file:
    lines = [line for line in file.readlines() if len(line.strip()) > 0 and 
             not line.strip().startswith('-')]

In [14]:
print(len(lines))

17975


In [10]:
print(re.match('\d{1,4}', '40000'))

<_sre.SRE_Match object; span=(0, 4), match='4000'>


In [21]:
data = {}
for index, line in enumerate(lines):
    if index % 5 == 0:
        if len(list(data.keys())) > 0:
            data_list.append(data)
            data = {}
        try:
            data['id'] = int(line)
        except:
            data['id'] = None
    if index % 5 == 1:
        data['district'] = line.strip()
    if index % 5 == 2:
        data['street'] = line.strip()
    if index % 5 == 3:
        data['subdivision'] = line.strip()
    if index % 5 == 4:
        try:
            data['price'] = int(line.strip())
        except:
            data['price'] = None
print(len(data_list))

3594


In [23]:
house_data = pd.DataFrame(data_list)

In [24]:
house_data.head()

,id,district,street,subdivision,price
0,1,罗湖,黄贝,安业花园,45000
1,2,罗湖,黄贝,安业馨园,56000
2,3,罗湖,黄贝,碧波花园,50500
3,4,罗湖,黄贝,碧中园,51600
4,5,罗湖,黄贝,赤龙坑住宅楼,51400


In [25]:
describe = house_data['price'].describe()
dict_describe = {'item': describe.index, 'value': describe.values}
df_describe = pd.DataFrame(dict_describe)
df_describe

,item,value
0,count,3594.000000
1,mean,61330.050083
2,std,22407.423764
3,min,16100.000000
4,25%,45000.000000
5,50%,56600.000000
6,75%,73500.000000
7,max,132000.000000


In [29]:
if not os.path.exists('./excel/'):
    os.makedirs('./excel/')

In [33]:
group_summary = house_data['price'].groupby(house_data['district']).describe()

In [35]:
import time
try:
    excel_name = 'shenzhen_house_{0}.xlsx'.format(
        time.strftime('%Y%m%d%H%M%S', time.localtime(time.time())))
    if not os.path.exists('./excel/'):
        os.makedirs('./excel/')
    excel_filepath = os.path.join('./excel/',
                                  excel_name)
    write = pd.ExcelWriter(excel_filepath)
    house_data.to_excel(write,
                        sheet_name='details',
                        index=False,
                        encoding='utf-8')
    df_describe.to_excel(write,
                         sheet_name='summary',
                         index=False,
                         encoding='utf-8')
    group_summary.to_excel(write,
                           sheet_name='district_summary',
                           index=True,
                           encoding='utf-8')
    write.save()
except Exception as e:
    print(e)